In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from dataclasses import dataclass
import utils
from torch_geometric_temporal import  DynamicGraphTemporalSignal,StaticGraphTemporalSignal
import networkx as nx
from torch_geometric.utils import from_networkx
import scipy
import sklearn
from tqdm import tqdm
import torch
from torch import nn
from torch.nn import functional as F
from torch_geometric_temporal.nn.recurrent import DCRNN,  GConvGRU, A3TGCN
from torch_geometric_temporal.nn.attention import STConv
from torchmetrics.classification import BinaryRecall
from torch_geometric.nn import global_max_pool
import timeit


In [ ]:
def plv_connectivity(sensors,data):
    """
    Parameters
    ----------
    sensors : INT
        DESCRIPTION. No of sensors used for capturing EEG
    data : Array of float 
        DESCRIPTION. EEG Data
    
    Returns
    -------
    connectivity_matrix : Matrix of float
        DESCRIPTION. PLV connectivity matrix
    connectivity_vector : Vector of flaot 
        DESCRIPTION. PLV connectivity vector
    """
    print("PLV in process.....")
    
    # Predefining connectivity matrix
    connectivity_matrix = np.zeros([sensors,sensors],dtype=float)
    
    # Computing hilbert transform
    data_points = data.shape[-1]
    data_hilbert = np.imag(scipy.signal.hilbert(data))
    phase = np.arctan(data_hilbert/data)
    
    # Computing connectivity matrix 
    for i in range(sensors):
        for k in range(sensors):
            connectivity_matrix[i,k] = np.abs(np.sum(np.exp(1j*(phase[i,:]-phase[k,:]))))/data_points
            
    # Computing connectivity vector
   # connectivity_vector = connectivity_matrix[np.triu_indices(connectivity_matrix.shape[0],k=1)] 
      
    # returning connectivity matrix and vector
    
    return connectivity_matrix

In [ ]:
def create_recordings_plv(npy_dataset_path,dst_path):
    patient_list = os.listdir(npy_dataset_path)
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)
    for patient in patient_list: # iterate over patient names
        patient_path = os.path.join(npy_dataset_path,patient)
        recording_list = os.listdir(patient_path)
        save_folder = os.path.join(dst_path,patient)
        if not os.path.exists(save_folder):
            os.mkdir(save_folder)
        for record in recording_list: # iterate over recordings for a patient
            recording_path = os.path.join(patient_path,record)
            data_array = np.load(recording_path) # load the recording
            starttime = timeit.default_timer()
            print(f'Calculating PLV for {record}')
            plv_array = plv_connectivity(data_array.shape[0],data_array)
            target_filename = os.path.join(save_folder,record)
            np.save(target_filename,plv_array)
            print("The time of calculation is :", timeit.default_timer() - starttime)

In [ ]:
create_recordings_plv('data/npy_data','data/plv_arrays')

In [ ]:

@dataclass
class SeizureDataLoader:
    npy_dataset_path :Path
    event_tables_path : Path
    plv_values_path : Path
    loso_patient : str = None
    sampling_f : int = 256
    seizure_lookback: int = 600
    sample_timestep: int = 5
    overlap: int = 0
    self_loops : bool = True
    shuffle=True
    """Class to prepare dataloaders for eeg seizure perdiction from stored files.

    Attributes:
        npy_dataset_path {Path} -- Path to folder with dataset preprocessed into .npy files.
        event_tables_path {Path} -- Path to folder with .csv files containing seizure events information for every patient.
        loso_patient {str} -- Name of patient to be selected for LOSO valdiation, specified in format "chb{patient_number}"",
        eg. "chb16". (default: {None}).
        samplin_f {int} -- Sampling frequency of the loaded eeg data. (default: {256}).
        seizure_lookback {int} -- Time horizon to sample pre-seizure data (length of period before seizure) in seconds. 
        (default: {600}).
        sample_timestep {int} -- Amounts of seconds analyzed in a single sample. (default: {5}).
        overlap {int} -- Amount of seconds overlap between samples. (default: {0}).
        self_loops {bool} -- Wheather to add self loops to nodes of the graph. (default: {True}).
        shuffle {bool} --  Wheather to shuffle training samples.


    """
    def _get_event_tables(self,patient_name : str) -> tuple[dict,dict]:
        """Read events for given patient into start and stop times lists from .csv extracted files."""

        event_table_list = os.listdir(self.event_tables_path)
        patient_event_tables = [os.path.join(self.event_tables_path,ev_table)
        for ev_table in event_table_list if patient_name in ev_table]
        patient_event_tables = sorted(patient_event_tables)
        patient_start_table = patient_event_tables[0] ## done terribly, but it has to be so for win/linux compat
        patient_stop_table = patient_event_tables[1]
        start_events_dict = pd.read_csv(patient_start_table).to_dict('index')
        stop_events_dict = pd.read_csv(patient_stop_table).to_dict('index')
        return start_events_dict,stop_events_dict
        
    def _get_recording_events(self,events_dict,recording) -> list[int]:
        """Read seizure times into list from event_dict"""
        recording_list = list(events_dict[recording+'.edf'].values())
        recording_events = [int(x) for x in recording_list if not np.isnan(x)]
        return recording_events


    def _get_graph(self,n_nodes: int) -> nx.Graph :
        """Creates Networx fully connected graph with self loops"""
        graph = nx.complete_graph(n_nodes)
        self_loops = [[node,node]for node in graph.nodes()]
        graph.add_edges_from(self_loops)
        return graph
    
    def _get_edge_weights_recording(self,plv_values: np.ndarray) ->np.ndarray:
        """Method that takes plv values for given recording and assigns them 
        as edge attributes to a fc graph."""
        graph = self._get_graph(plv_values.shape[0])
        garph_dict = {}
        for edge in graph.edges():
            e_start,e_end = edge
            garph_dict[edge] = {'plv':plv_values[e_start,e_end]}
        nx.set_edge_attributes(graph, garph_dict)
        edge_weights = from_networkx(graph).plv.numpy()
        return edge_weights
    
    def _get_edges(self):
        """Method to assign edge attributes. Has to be called AFTER get_dataset() method."""
        graph = self._get_graph(self._features.shape[1])
        edges = np.expand_dims(from_networkx(graph).edge_index.numpy(),axis=0)
        edges_per_sample_train = np.repeat(edges,repeats =self._features.shape[0],axis=0)
        edges_per_sample_val = np.repeat(edges,repeats =self._val_features.shape[0],axis=0)
        self._edges = edges_per_sample_train
        self._val_edges = edges_per_sample_val
       
        
    def _get_labels_features_edge_weights(self):
        """Prepare features, labels, time labels and edge wieghts for training and 
        optionally validation data."""
        patient_list = os.listdir(self.npy_dataset_path)
        for patient in patient_list: # iterate over patient names
            event_tables = self._get_event_tables(patient) # extract start and stop of seizure for patient 
            patient_path = os.path.join(self.npy_dataset_path,patient)
            recording_list = os.listdir(patient_path)
            for record in recording_list: # iterate over recordings for a patient
                recording_path = os.path.join(patient_path,record)
                record_id = record.split('.npy')[0] #  get record id
                start_event_tables = self._get_recording_events(event_tables[0],record_id) # get start events
                stop_event_tables = self._get_recording_events(event_tables[1],record_id) # get stop events
                data_array = np.load(recording_path) # load the recording

                plv_edge_weights = np.expand_dims(
                    self._get_edge_weights_recording(
                        np.load(os.path.join(self.plv_values_path,patient,record))
                        #np.random.uniform(size=(18,18))
                    #plv_connectivity(data_array.shape[0],data_array)
                ),
                axis = 0
                )

                ##TODO add a gateway to reject seizure periods shorter than lookback
                # extract timeseries and labels from the array
                features,labels,time_labels = utils.extract_training_data_and_labels(
                    data_array,
                    start_event_tables,
                    stop_event_tables,
                    fs = self.sampling_f,
                    seizure_lookback = self.seizure_lookback,
                    sample_timestep = self.sample_timestep,
                    overlap = self.overlap,
                )
                
                time_labels = time_labels.astype(np.int32)
                labels = labels.reshape((labels.shape[0],1)).astype(np.float32)
            
                if patient == self.loso_patient:
                    try:
                        self._val_features = np.concatenate((self._val_features, features))
                        self._val_labels = np.concatenate((self._val_labels, labels))
                        self._val_time_labels = np.concatenate((self._val_time_labels , time_labels))
                        self._val_edge_weights = np.concatenate((
                            self._val_edge_weights,
                            np.repeat(plv_edge_weights,features.shape[0],axis=0)
                            ))
                    except:
                        self._val_features = features
                        self._val_labels = labels
                        self._val_time_labels = time_labels
                        self._val_edge_weights = np.repeat(plv_edge_weights,features.shape[0],axis=0)
                else:
                    try:
                        self._features = np.concatenate((self._features, features))
                        self._labels = np.concatenate((self._labels, labels))
                        self._time_labels = np.concatenate((self._time_labels , time_labels))
                        self._edge_weights = np.concatenate((
                            self._edge_weights,
                            np.repeat(plv_edge_weights,features.shape[0],axis=0)
                        ))
                       
                    except:
                        print("Creating initial attributes")
                        self._features = features
                        self._labels = labels
                        self._time_labels = time_labels
                        self._edge_weights = np.repeat(plv_edge_weights,features.shape[0],axis=0)
                
        if self.shuffle is True:
            shuffled_features, shuffled_labels, shuffled_time_labels, shuffled_edge_weights = sklearn.utils.shuffle(self._features,self._labels,self._time_labels,self._edge_weights)
            self._features = shuffled_features
            self._labels = shuffled_labels
            self._time_labels = shuffled_time_labels
            self._edge_weights = shuffled_edge_weights
    # TODO define a method to create edges and calculate plv to get weights
    def get_dataset(self) -> DynamicGraphTemporalSignal:

        """Creating graph data iterators. The iterator yelds dynamic, weighted and undirected graphs
        containing self loops. Every node represents one electrode in EEG. The graph is fully connected,
        edge weights are calculated for every EEG recording as PLV between channels (edge weight describes 
        the "strength" of connectivity between two channels in a recording). Node features are values of 
        channel voltages in time. Features are of shape [nodes,features,timesteps].

        Returns:
            train_dataset {DynamicGraphTemporalSignal} -- Training data iterator.
            valid_dataset {DynamicGraphTemporalSignal} -- Validation data iterator (only if loso_patient is
            specified in class constructor).
        """
        ### TODO rozkminić o co chodzi z tym całym time labels - na razie wartość liczbowa która tam wchodzi
        ### to shape atrybutu time_labels
        
        self._get_labels_features_edge_weights()
        self._get_edges()
        
        train_dataset = DynamicGraphTemporalSignal(
        self._edges, self._edge_weights, self._features, self._labels, time_labels = self._time_labels
        )
        if self.loso_patient:
            val_dataset = DynamicGraphTemporalSignal(
            self._val_edges, self._val_edge_weights, self._val_features, self._val_labels, time_labels = self._val_time_labels
            )
            return train_dataset, val_dataset

        return train_dataset
        
                

        

In [ ]:
TIMESTEP = 5
OVERLAP = 0
SFREQ = 256
dataloader = SeizureDataLoader(
    npy_dataset_path=Path('data/npy_data'),
    event_tables_path=Path('data/event_tables'),
    plv_values_path=Path('data/plv_arrays'),
    loso_patient='chb16',
    sampling_f=SFREQ,
    seizure_lookback=600,
    sample_timestep= TIMESTEP,
    overlap=OVERLAP,
    self_loops=True
    )

In [ ]:
train_loader,val_loader=dataloader.get_dataset()

In [ ]:
y_list =[]
for snapshot in train_loader:
    print(snapshot.x.shape)
    y_list.append(int(snapshot.y.numpy()[0]))

In [ ]:
y_list_val =[]
for snapshot in val_loader:
    y_list_val.append(int(snapshot.y.numpy()[0]))

In [ ]:
np.unique(np.array(y_list_val),return_counts=True)

In [ ]:
np.unique(np.array(y_list),return_counts=True)

In [ ]:
class RecurrentGCN(torch.nn.Module):
    def __init__(self, timestep,sfreq, n_nodes=18):
        super(RecurrentGCN, self).__init__()
        out_features = 64
        self.recurrent_1 = DCRNN(timestep*sfreq,out_features,1)
        self.fc1 = torch.nn.Linear(out_features*n_nodes, 1)
        self.flatten = torch.nn.Flatten(start_dim=0)
    def forward(self, x, edge_index, edge_weight):
        x = torch.squeeze(x)
        x = F.normalize(x,dim=1)
        h = self.recurrent_1(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.flatten(h)
        h = self.fc1(h)
        return h

In [ ]:
class ModelStconv(torch.nn.Module):
    def __init__(self, timestep,sfreq, n_nodes=18):
        super(RecurrentGCN, self).__init__()
        out_features = 64
        self.recurrent_1 =  STConv(
            num_nodes=n_nodes,
            in_channels = 1,
            hidden_channels=64,
            out_channels=out_features,
            kernel_size=3,
            K=1)
        self.fc1 = torch.nn.Linear(1469952, 1)
        self.flatten = torch.nn.Flatten(start_dim=0)
    def forward(self, x, edge_index, edge_weight):
        x = F.normalize(x,dim=1)
        x = x[None,:]
        x = x.reshape([x.shape[0],x.shape[3],x.shape[1],x.shape[2]])
        h = self.recurrent_1(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.flatten(h)
        h = self.fc1(h)
        return h

In [ ]:
class A3TRecurrent(torch.nn.Module):
    def __init__(self,node_features, timestep,sfreq, n_nodes=18):
        super(A3TRecurrent, self).__init__()
        out_features = 64
        self.recurrent_1 =  A3TGCN(node_features,out_features,timestep*sfreq)
        self.fc1 = torch.nn.Linear(1152, 1)
        self.flatten = torch.nn.Flatten(start_dim=0)
    def forward(self, x, edge_index, edge_weight):
        x = F.normalize(x,dim=1)
        h = self.recurrent_1(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.flatten(h)  
        h = self.fc1(h)
        return h

In [ ]:
from sklearn import metrics
def get_accuracy(y_true, y_prob):
    """Binary accuracy calculation"""
    y_prob = np.array(y_prob)
    y_prob = np.where(y_prob <= 0.0, 0, y_prob)
    y_prob = np.where(y_prob > 0.0, 1, y_prob)

    accuracy = metrics.accuracy_score(y_true, y_prob)
    return accuracy

In [ ]:
## TODO 
## 1. Implement kfold cross validation .get_dataset() will return also a kfold cross validation split?
## 2. Think of time series augmentation techniques
## 3. Try to add a feature balancing between ictal and iterictal periods.
## 4.Try to run the algorithm on list of patients shown in the articles.
## 5. !Why there is no patient 15 and 20 in the dataset? !!

In [ ]:
for patient in os.listdir("data/reduced_npy_data"):
    TIMESTEP = 5
    OVERLAP = 0
    SFREQ = 256
    dataloader = SeizureDataLoader(
        npy_dataset_path=Path('data/npy_data'),
        event_tables_path=Path('data/event_tables'),
        plv_values_path=Path('data/plv_arrays'),
        loso_patient=patient,
        sampling_f=SFREQ,
        seizure_lookback=600,
        sample_timestep= TIMESTEP,
        overlap=OVERLAP,
        self_loops=True
        )
    train_loader,val_loader=dataloader.get_dataset()
    device = torch.device("cpu")
    model = RecurrentGCN(TIMESTEP,SFREQ).to(device)
    loss_fn =  nn.BCEWithLogitsLoss(pos_weight=torch.full([1], 15))
    optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
    recall = BinaryRecall(threshold=0.5)
    model.train()

    for epoch in tqdm(range(10)):
        
        preds = []
        ground_truth = []
        epoch_loss = 0.0
        epoch_loss_valid = 0.0
        preds_valid = []
        ground_truth_valid = []
        model.train()
        for time, snapshot in enumerate(train_loader):
            
                y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
                ##loss
        
                loss = loss_fn(y_hat,snapshot.y)
                epoch_loss += loss
                ## get preds & gorund truth
                preds.append(y_hat.detach().numpy()[0])
                ground_truth.append(snapshot.y.numpy()[0])
                ##backprop
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        ## calculate acc
    
        sensitivity = recall(torch.FloatTensor(preds),torch.FloatTensor(ground_truth))
        print(f'Epoch: {epoch}',f'Epoch sensitivity: {sensitivity}', f'Epoch loss: {epoch_loss.detach().numpy()}')
        model.eval()
        for time_valid, snapshot_valid in enumerate(val_loader):

        # if time_valid == 171 or time_valid == 169:
                y_hat_val = model(snapshot_valid.x,snapshot_valid.edge_index, snapshot_valid.edge_attr)
                loss_valid = loss_fn(y_hat_val,snapshot_valid.y)
                epoch_loss_valid += loss_valid
                preds_valid.append(y_hat_val.detach().numpy()[0])
                ground_truth_valid.append(snapshot_valid.y.numpy()[0])
                
        
        val_sensitivity = recall(torch.FloatTensor(preds_valid),torch.FloatTensor(ground_truth_valid))
        print(f'Epoch: {epoch}',f'Epoch val_sensitivity: {val_sensitivity}', f'Epoch val_loss: {epoch_loss_valid.detach().numpy()}')

In [ ]:
np.where(np.array(ground_truth) == 1)[0] 

In [ ]:
np.equal(np.where(np.array(ground_truth) == 1)[0],np.where(np.array(preds) >0)[0] )

In [ ]:
np.where(np.array(preds) >0)[0]